In [ ]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib qt
import numpy as np
import cv2 as cv
import os
from typing import List
import matplotlib.pyplot as plt

In [ ]:
def custom_pyramid(img, scales: List[float]) -> list:
    img_copy = img.copy()

    resized_imgs = list()

    # generamos nuestra "piramide" de tamaños arbitrarios
    for scale in scales:
        resized_imgs.append(cv.resize(img_copy, None, fx=scale, fy=scale))
        
    return resized_imgs

In [ ]:
# Ruta de la carpeta con las imágenes
directorio = "./images"

# Lista para guardar las imágenes en gris
imagenes_gris = []

# Recorremos los archivos del directorio
for archivo in os.listdir(directorio):
    ruta_completa = os.path.join(directorio, archivo)

    # Verificamos que sea un archivo y termine en una extensión de imagen
    if os.path.isfile(ruta_completa) and archivo.lower().endswith(('.png', '.jpg')):
        img = cv.imread(ruta_completa, cv.IMREAD_GRAYSCALE)
        if img is not None:
            imagenes_gris.append(img)
        else:
            print(f"No se pudo leer la imagen: {archivo}")

print(f"Se leyeron {len(imagenes_gris)} imágenes en blanco y negro")

In [ ]:
img_aux = []
for img4 in imagenes_gris:
    img_aux.extend(custom_pyramid(img4, np.linspace(0.2, 2, 10)))
imagenes_gris = img_aux

In [ ]:
template = cv.imread('./template/pattern.png',0)

In [ ]:
template_list = custom_pyramid(template, np.linspace(0.2, 2, 10))

In [ ]:
len(template_list)

In [56]:
def match_template(img3, template):
    img_copy1 = img3.copy()
    template_copy = template.copy()
    w, h = template.shape[::-1]

    # --- Desenfoque Gaussiano ---
    img_blur = cv.GaussianBlur(img_copy1, (5,5), 0)
    template_blur = cv.GaussianBlur(template_copy, (5,5), 0)

    # --- Bordes con Canny ---
    img_edges = cv.Canny(img_blur, 50, 150)
    template_edges = cv.Canny(template_blur, 50, 150)


    # print(f"Buscando plantilla de tamaño {template_edges.shape[0]}x{template_edges.shape[1]} en imagen de tamaño {img_edges.shape[0]}x{img_edges.shape[1]}")
    res = cv.matchTemplate(img_edges, template_edges, cv.TM_CCOEFF_NORMED)
    # Fijamos un umbral de "coincidencia"

    threshold = 0.1
    flag = False
    loc = np.where( res >= threshold)
    for pt in zip(*loc[::-1]):
        cv.rectangle(img_copy1, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
        flag = True
    
    if flag:
        return img_copy1
    else:
        return None

In [57]:
def search_template(img1, template_list):
    img_copy5 = img1.copy()

    result = []
    for i, template in enumerate(template_list):
        try:
            img9 = match_template(img_copy5, template)
            if img9 is not None:
                result.append(img9)
            # cv.imshow(f'Template Level {i}', img_copy)
            # cv.waitKey(0)
        except Exception as e:
            # print(f"Error processing template level {i}: {e}")
            continue
    return result

In [58]:
def run_all_img(img_list, template_list):
    result = []
    for img2 in img_list:
        result.extend(search_template(img2, template_list))
    return result

In [59]:
r = run_all_img(imagenes_gris, template_list)

In [60]:
len(r)

158

In [64]:
cv.imshow(f'Template Level', r[3])